# Day 21
## Part 1

In [43]:
def die():
    while True:
        yield from range(1, 101)
        
def rolls():
    d = die()
    while True:
        yield next(d) + next(d) + next(d)
        
def move(p, n):
    return (p - 1 + n) % 10 + 1

def part_1(player1, player2):
    r = rolls()
    n_rolls = 0
    scores = [0, 0]
    positions = [player1, player2]
    while True:
        for i in range(2):
            positions[i] = move(positions[i], next(r))
            scores[i] += positions[i]
            n_rolls += 3
            if scores[i] >= 1000:
                return scores[0 if i else 1] * n_rolls
            
assert part_1(4, 8) == 739785

In [44]:
part_1(8, 9)

512442

## Part 2
Ha ha! I thought part 1 was a bit easy for day 21.

OK, thinking about it, there are only a limited number of states, representing
- player 1's position (10 possibilities)
- player 2's position (10)
- player 1's score (21)
- player 2's score (21)
- whose turn it is (2)

which is 80 odd thousand - a lot, but tractable. Try a cached recursive solution and see if that falls over.

The die is rolled three times, rather than simulate that keep a total of how often each total occurs.

In [45]:
from collections import Counter
from functools import cache
import itertools

possible_rolls = Counter(sum(ds) for ds in itertools.product(range(1, 4), repeat=3))
possible_rolls

Counter({3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1})

In [46]:
@cache
def multiverse(p1, p2, s1, s2, turn):
    player1_wins = 0
    player2_wins = 0
    for roll in possible_rolls:
        if turn == 1:
            p = move(p1, roll)
            if s1 + p >= 21:
                player1_wins += possible_rolls[roll]
            else: 
                ws = multiverse(p, p2, s1 + p, s2, 2)
                player1_wins += ws[0] * possible_rolls[roll]
                player2_wins += ws[1] * possible_rolls[roll]
        elif turn == 2:
            p = move(p2, roll)
            if s2 + p >= 21:
                player2_wins += possible_rolls[roll]
            else: 
                ws = multiverse(p1, p, s1, s2 + p, 1)
                player1_wins += ws[0] * possible_rolls[roll]
                player2_wins += ws[1] * possible_rolls[roll]
    return (player1_wins, player2_wins)

In [47]:
multiverse(4, 8, 0, 0, 1)

(444356092776315, 341960390180808)

In [48]:
multiverse(8, 9, 0, 0, 1)

(346642902541848, 262939886779945)

In [49]:
max(_)

346642902541848

In [50]:
%%timeit 
multiverse(8, 9, 0, 0, 1)

182 ns ± 5.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
